In [0]:
# ——————————————————————————————
#  COMMON SETUP
# ——————————————————————————————
env           = spark.conf.get("pipeline.env")  # např. "dev" nebo "prod"
catalog       = "principal_lab_db"
silver_schema = f"{env}_silver"
gold_schema   = f"{env}_gold"

# Nastav katalog a schéma
spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {silver_schema}")

In [0]:
# ——————————————————————————————
#  CREATE GOLD VIEW
# ——————————————————————————————
gold_digger_view = f"""
CREATE OR REPLACE VIEW {catalog}.{env}_gold.agent_performance_view AS
SELECT
    a.agent_id,
    a.first_name,
    a.last_name,
    COUNT(DISTINCT p.policy_id) AS num_policies,
    ROUND(SUM(pt.premium_amount), 2) AS total_premium,
    COUNT(DISTINCT c.claim_id) AS num_claims,
    ROUND(SUM(c.amount), 2) AS total_claim_amount
FROM {catalog}.{silver_schema}.dim_agents a
LEFT JOIN {catalog}.{silver_schema}.dim_policies p USING (agent_id)
LEFT JOIN {catalog}.{silver_schema}.fact_premium_transactions pt USING (policy_id)
LEFT JOIN {catalog}.{silver_schema}.fact_claims c USING (policy_id)
GROUP BY a.agent_id, a.first_name, a.last_name
"""


In [0]:
spark.sql(gold_digger_view)